In [ ]:
# Base

@decorate
def target():
    print('running target()')

def target():
    print('running target')

In [ ]:
# 7-1
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

print(target())
print(target)

In [1]:
# 7-2
registry = []  # <1>

def register(func):  # <2>
    print('running register(%s)' % func)  # <3>
    registry.append(func)  # <4>
    return func  # <5>

@register  # <6>
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():  # <7>
    print('running f3()')

def main():  # <8>
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

main()

running register(<function f1 at 0x1053f1360>)
running register(<function f2 at 0x1053f1480>)
running main()
registry -> [<function f1 at 0x1053f1360>, <function f2 at 0x1053f1480>]
running f1()
running f2()
running f3()


In [2]:
import registration


running register(<function f1 at 0x1053f1120>)
running register(<function f2 at 0x1053f11b0>)


In [3]:
# 7-3
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity
    

class Order:

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total * .07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)

In [15]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

# print(Order(joe, long_order, best_promo))
print(Order(joe, banana_cart, best_promo))
print(Order(ann, cart, best_promo))

<Order total: 30.00 due: 28.50>
<Order total: 42.00 due: 39.90>


In [16]:
# 7-4
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [17]:
b = 6
f1(3)

3
6


In [18]:
# 7-5
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [19]:
# 7-6
from dis import dis
dis(f1)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [20]:
# 7-7
dis(f2)

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  6          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


In [22]:
# 7-8
class Averager():

    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)
    
avg = Averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [25]:
# 7-9
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

# 7-10
avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [26]:
# 7-11
print(avg.__code__.co_varnames)
print(avg.__code__.co_freevars)

('new_value', 'total')
('series',)


In [27]:
# 7-12
print(avg.__code__.co_freevars)
print(avg.__closure__)
print(avg.__closure__[0].cell_contents)

('series',)
(<cell at 0x1055be890: list object at 0x109a66800>,)
[10, 11, 12]


In [28]:
# 7-13
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [29]:
# 7-14
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

avg = make_averager()
avg(10)

10.0

In [30]:
# 7-15
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [32]:
# clockdeco_demo.py

import time

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))


**************************************** Calling snooze(.123)
[0.12424058s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000038s] factorial(1) -> 1
[0.00006742s] factorial(2) -> 2
[0.00009200s] factorial(3) -> 6
[0.00014979s] factorial(4) -> 24
[0.00016008s] factorial(5) -> 120
[0.00018229s] factorial(6) -> 720
6! = 720


In [33]:
factorial.__name__

'clocked'

In [34]:
# 7-17
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))


**************************************** Calling snooze(.123)
[0.12811954s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000029s] factorial(1) -> 1
[0.00003146s] factorial(2) -> 2
[0.00023254s] factorial(3) -> 6
[0.00024858s] factorial(4) -> 24
[0.00039167s] factorial(5) -> 120
[0.00058350s] factorial(6) -> 720
6! = 720


In [38]:
# 7-18

@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__=='__main__':
    print(fibonacci(6))

[0.00000095s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00002098s] fibonacci(2) -> 1
[0.00000095s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000095s] fibonacci(1) -> 1
[0.00000405s] fibonacci(2) -> 1
[0.00000787s] fibonacci(3) -> 2
[0.00003314s] fibonacci(4) -> 3
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000095s] fibonacci(1) -> 1
[0.00000310s] fibonacci(2) -> 1
[0.00000691s] fibonacci(3) -> 2
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000405s] fibonacci(2) -> 1
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000381s] fibonacci(2) -> 1
[0.00000787s] fibonacci(3) -> 2
[0.00001407s] fibonacci(4) -> 3
[0.00002503s] fibonacci(5) -> 5
[0.00006223s] fibonacci(6) -> 8
8


In [39]:
# 7-19
import functools

@functools.lru_cache() # <1>
@clock  # <2>
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__=='__main__':
    print(fibonacci(6))

[0.00000095s] fibonacci(0) -> 0
[0.00000095s] fibonacci(1) -> 1
[0.00003767s] fibonacci(2) -> 1
[0.00000095s] fibonacci(3) -> 2
[0.00004721s] fibonacci(4) -> 3
[0.00000000s] fibonacci(5) -> 5
[0.00005603s] fibonacci(6) -> 8
8


In [40]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

# 7-20
print(htmlize({1, 2, 3}))
print(htmlize(abs))
print(htmlize('Heimlich & Co.\n- a game'))
print(htmlize(42))
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>{1, 2, 3}</pre>
<pre>&lt;built-in function abs&gt;</pre>
<pre>&#x27;Heimlich &amp; Co.\n- a game&#x27;</pre>
<pre>42</pre>
<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>


In [41]:
htmlize(['alpha', 66, {3, 2, 1}])

'<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>'

In [ ]:
# 7-21
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch  # <1>
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)  # <2>
def _(text):            # <3>
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)  # <4>
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)  # <5>
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

In [42]:
# 7-22
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x1053f1240>)
running main()
registry -> [<function f1 at 0x1053f1240>]
running f1()


In [43]:
# 7-22
registry = set()  # <1>

def register(active=True):  # <2>
    def decorate(func):  # <3>
        print('running register(active=%s)->decorate(%s)'
              % (active, func))
        if active:   # <4>
            registry.add(func)
        else:
            registry.discard(func)  # <5>

        return func  # <6>
    return decorate  # <7>

@register(active=False)  # <8>
def f1():
    print('running f1()')

@register()  # <9>
def f2():
    print('running f2()')

def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x109a824d0>)
running register(active=True)->decorate(<function f2 at 0x1053f1360>)


In [45]:
registry

{<function __main__.f2()>}

In [48]:
# 7-24
print(register)
print(registry)
print(register()(f3))
print(registry)
print(register(active=False)(f2))
print(registry)

<function register at 0x109a83250>
{<function f3 at 0x109a1e200>, <function f2 at 0x1053f1360>}
running register(active=True)->decorate(<function f3 at 0x109a1e200>)
<function f3 at 0x109a1e200>
{<function f3 at 0x109a1e200>, <function f2 at 0x1053f1360>}
running register(active=False)->decorate(<function f2 at 0x1053f1360>)
<function f2 at 0x1053f1360>
{<function f3 at 0x109a1e200>}


In [49]:
# 7-25
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):  # <1>
    def decorate(func):      # <2>
        def clocked(*_args): # <3>
            t0 = time.time()
            _result = func(*_args)  # <4>
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)  # <5>
            result = repr(_result)  # <6>
            print(fmt.format(**locals()))  # <7>
            return _result  # <8>
        return clocked  # <9>
    return decorate  # <10>

if __name__ == '__main__':

    @clock()  # <11>
    def snooze(seconds):
        time.sleep(seconds)

    for i in range(3):
        snooze(.123)

[0.12810993s] snooze(0.123) -> None
[0.14249587s] snooze(0.123) -> None
[0.12805319s] snooze(0.123) -> None


In [51]:
# 7-26
import time

@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.1251230239868164s
snooze: 0.12407493591308594s
snooze: 0.12682318687438965s


In [53]:
# 7-27
import time

@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.124s
snooze(0.123) dt=0.125s
snooze(0.123) dt=0.128s
